<a href="https://colab.research.google.com/github/alaa-alt/ITI/blob/main/Recommendation_Systems/recommender_systems_lab3_Alaa_Abdelmonsef_Elkaffas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

$$ ITI \space AI-Pro: \space Intake \space 45 $$
$$ Recommender \space Systems $$
$$ Lab \space no. \space 3 $$

# `01` Import Necessary Libraries

## `i` Default Libraries

In [ ]:
import numpy as np
import pandas as pd

## `ii` Additional Libraries
Add imports for additional libraries you used throughout the notebook

In [ ]:
!pip install numpy==1.24.3
!pip install scikit-surprise

In [ ]:
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import SVD
from surprise import accuracy

----------------------------

# `02` Load Data

Load `songsDataset.csv` file into a dataframe

In [ ]:
data = pd.read_csv("/content/songsDataset.csv")
data.head()

,userID,songID,rating
0,0,90409,5
1,4,91266,1
2,5,8063,2
3,5,24427,4
4,5,105433,4


--------------------------

# `03` Matrix Factorization using Gradient Descent

Practice for Matrix Factorization Implementation

**Matrix Factorization Mathematical Derivation**

We know that the matrix factorization breaks the rating matrix $R$ into two matrices $U \in \mathbb{R}^{\#users \times K}$ and $M \in \mathbb{R}^{K \times \#items}$ where K represent the latent space dimensionality.

$R$ can then be approximated through the following equation:

$$
\mathbf{R} \approx \mathbf{U} \times \mathbf{M} = \hat{\mathbf{R}}
$$

The error, incorporating the regularization term, is calculated as follows:

$$
e_{ij}^2 = (r_{ij} - \sum_{k=1}^K{u_{ik}m_{kj}})^2 + \frac{\beta}{2} \sum_{k=1}^K{(||U||^2 + ||M||^2)}
$$

In order to be able to use Stochastic Gradient Descent (SGD) to optimize $U$ and $M$, we need to find the partial derivatives of the error function with respect to both $u_{ik}$ and $m_{kj}$. The partial derivatives can be derived as follows:

$$
\frac{\partial}{\partial u_{ik}}e_{ij}^2 = -2(r_{ij} - \hat{r}_{ij})(m_{kj}) + \frac{\beta}{2} \times (2 u_{ik}) = -2 e_{ij} m_{kj} + \beta u_{ik}
$$

$$
\frac{\partial}{\partial m_{ik}}e_{ij}^2 = -2(r_{ij} - \hat{r}_{ij})(u_{ik}) + \frac{\beta}{2} \times (2 m_{kj}) = -2 e_{ij} u_{ik} + \beta m_{kj}
$$

Thus the update rules will be:

$$
u'_{ik} = u_{ik} + \alpha \frac{\partial}{\partial u_{ik}}e_{ij}^2 = u_{ik} - \alpha(-2 e_{ij} m_{kj} + \beta u_{ik} ) = u_{ik} + \alpha(2 e_{ij} m_{kj} - \beta u_{ik} )
$$

$$
m'_{kj} = m_{kj} + \alpha \frac{\partial}{\partial m_{kj}}e_{ij}^2 = m_{kj} - \alpha(-2 e_{ij} u_{ik} + \beta m_{kj} ) = m_{kj} + \alpha(2 e_{ij} u_{ik} - \beta m_{kj} )
$$

## `0` Construct Utility Matrix from the Data

In [ ]:
utility_matrix = data.pivot(index='userID', columns='songID', values='rating')
utility_matrix = utility_matrix.fillna(0)
utility_matrix

songID,2263,2726,3785,8063,12709,13859,16548,17029,19299,19670,...,113954,119103,120147,122065,123176,125557,126757,131048,132189,134732
userID,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199976,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0
199980,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
199988,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## `i` Matrices Initialization

Initialize the two random weights matrices $U$ and $M$ (Try $K=3$)

**Note**: Refer to the next cell for the dimensions of $U$ and $M$

**Hine**: You may use a function from `numpy.random` module (see, [Documentation](https://numpy.org/doc/stable/reference/random/index.html))

In [ ]:
num_users = data['userID'].nunique()
num_items = data['songID'].nunique()
U = np.random.rand(num_users, 3)
M = np.random.rand(3, num_items)

## `ii` Define a Function to Implement Matrix Factorization

**Function Parameters**:
- `R`: Utility Matrix [of shape: (no. of users, no. of items)]
- `U`: User Latent Features Array [of shape: (no. of users, K)]
- `M`: Items Latent Features Array [of shape: (K, no. of items)]
- `epochs`: No. of training epochs
- `lr`: Learning rate (alpha)
- `beta`: Regularization Parameter

**Function Output**:
- `U`: Optimized User Latent Features Array
- `M`: Optimized Items Latent Features Array

**Main Procedures**:
1. Calculate predicted ratings
2. Calculate MSE Error
3. Calculate gradients
4. Update $U$ and $M$


In [ ]:
def matrix_factorization(R: np.ndarray, U: np.ndarray, M: np.ndarray, epochs: int, lr: float, beta: float):
    """
    Function Parameters:
    - `R`: Utility Matrix [of shape: `(no. of users, no. of items)`]
    - `U`: User Latent Features Array [of shape: `(no. of users, K)`]
    - `M`: Items Latent Features Array [of shape: `(K, no. of items)`]
    - `epochs`: No. of training epochs
    - `lr`: Learning rate (alpha)
    - `beta`: Regularization Parameter

    Function Output:
    - `U`: Optimized User Latent Features Array
    - `M`: Optimized Items Latent Features Array
    """

    # Confirm that no. of features is consistent between U and M
    assert U.shape[1] == M.shape[0], f'U and M must have consistent K. Found K={U.shape[1]} for U and K={M.shape[0]} for M'

    # Extract No. of Features (K)
    K = U.shape[1]

    # Define the Epochs loop
    for epoch in range(epochs):
        # Loop over every element in R
        for i in range(R.shape[0]):  # Loop over each user
            for j in range(R.shape[1]):  # Loop over each item
                if R[i, j] > 0:  # Only proceed if the current interaction (i, j) is not missing
                    eij = R[i, j] - np.dot(U[i, :], M[:, j])  # Calculate the error in prediction
                    for k in range(K):  # Loop over each latent feature dimension
                        # Update Rules for both U and M:
                        U[i, k] += lr * (2 * eij * M[k, j] - beta * U[i, k])
                        M[k, j] += lr * (2 * eij * U[i, k] - beta * M[k, j])
        ## Error Calculation ##
        e_last = e if epoch > 0 else 100000000
        e = 0 # Initialize a variable to accumelate the errors
        for i in range(R.shape[0]):  # Loop over each user
            for j in range(R.shape[1]):  # Loop over each item
                if R[i, j] > 0:  # Only proceed if the current interaction (i, j) is not missing
                    first_part = (R[i, j] - np.dot(U[i, :], M[:, j])) ** 2  # Calculate the first part of the error
                    second_part = 0  # Initialize a variable to accumulate the second part of the error

                    for k in range(K):  # Loop over each latent feature dimension
                        second_part += (beta / 2) * (U[i, k] ** 2 + M[k, j] ** 2)
        print(f'Epoch {epoch+1}/{epochs}: Total Error = {e}')

        if e < 0.001 or e_last-e < 0.001: # Stop if error is so small or improvement is not significant
            break

    return U, M


### `[Bonus]` Vectorized Error Calculation

Can the error calculation part be vectorized to get rid of for loops?

If you would like a challenge, try to redefine the function in the next cell with a vectorized error calculation.

In [ ]:
def matrix_factorization(R: np.ndarray, U: np.ndarray, M: np.ndarray, epochs: int, lr: float, beta: float):
    """
    Function Parameters:
    - `R`: Utility Matrix [of shape: `(no. of users, no. of items)`]
    - `U`: User Latent Features Array [of shape: `(no. of users, K)`]
    - `M`: Items Latent Features Array [of shape: `(K, no. of items)`]
    - `epochs`: No. of training epochs
    - `lr`: Learning rate (alpha)
    - `beta`: Regularization Parameter

    Function Output:
    - `U`: Optimized User Latent Features Array
    - `M`: Optimized Items Latent Features Array
    """

    # Confirm that no. of features is consistent between U and M
    assert U.shape[1] == M.shape[0], f'U and M must have consistent K. Found K={U.shape[1]} for U and K={M.shape[0]} for M'

    K = U.shape[1]  # Extract number of latent features
    mask = (R > 0)   # Create a mask to identify non-zero elements

    for epoch in range(epochs):
        # Compute error matrix only for non-missing values
        prediction = np.dot(U, M)
        error_matrix = (R - prediction) * mask

        # Compute gradient updates
        U += lr * (2 * np.dot(error_matrix, M.T) - beta * U)
        M += lr * (2 * np.dot(U.T, error_matrix) - beta * M)

        # Vectorized Error Calculation
        squared_error = np.square(error_matrix)  # Element-wise square
        regularization = (beta / 2) * (np.sum(np.square(U)) + np.sum(np.square(M)))
        e = np.sum(squared_error) + regularization

        print(f'Epoch {epoch+1}/{epochs}: Total Error = {e:.4f}')

        if e < 0.001 or (epoch > 0 and e_last - e < 0.001):
            break
        e_last = e

    return U, M


## `iii` Use the Function to to Optimize the $U$ and $V$

In [ ]:
U, M = matrix_factorization(R=utility_matrix.values, U=U, M=M, epochs=7000, lr=0.0005, beta=0.02)

Streaming output truncated to the last 5000 lines.
Epoch 2001/7000: Total Error = 2535.9741
Epoch 2002/7000: Total Error = 2535.4464
Epoch 2003/7000: Total Error = 2534.9193
Epoch 2004/7000: Total Error = 2534.3928
Epoch 2005/7000: Total Error = 2533.8668
Epoch 2006/7000: Total Error = 2533.3414
Epoch 2007/7000: Total Error = 2532.8167
Epoch 2008/7000: Total Error = 2532.2925
Epoch 2009/7000: Total Error = 2531.7689
Epoch 2010/7000: Total Error = 2531.2458
Epoch 2011/7000: Total Error = 2530.7234
Epoch 2012/7000: Total Error = 2530.2015
Epoch 2013/7000: Total Error = 2529.6803
Epoch 2014/7000: Total Error = 2529.1596
Epoch 2015/7000: Total Error = 2528.6394
Epoch 2016/7000: Total Error = 2528.1199
Epoch 2017/7000: Total Error = 2527.6009
Epoch 2018/7000: Total Error = 2527.0826
Epoch 2019/7000: Total Error = 2526.5647
Epoch 2020/7000: Total Error = 2526.0475
Epoch 2021/7000: Total Error = 2525.5308
Epoch 2022/7000: Total Error = 2525.0147
Epoch 2023/7000: Total Error = 2524.4992
Epoch 

## `iv` Recommend top-K Songs

Recommend top-K ($K=5$) songs for user ($userID=199988$)

Note: Make sure to filter songs they already rated before

In [ ]:
def recommend_songs(U, M, R, user_id, user_ids, K=5):

    if user_id not in user_ids:
        raise ValueError(f"User ID {user_id} not found in dataset.")

    user_index = np.where(user_ids == user_id)[0][0]
    R_pred = np.dot(U, M)
    user_ratings = R[user_index, :]
    R_pred[user_index, user_ratings > 0] = -np.inf
    top_k_indices = np.argsort(R_pred[user_index, :])[-K:][::-1]
    top_k_ratings = R_pred[user_index, top_k_indices]

    return top_k_indices, top_k_ratings
user_ids = np.array(utility_matrix.index)
top_songs, top_ratings = recommend_songs(U, M, utility_matrix.values, user_id=199988, user_ids=user_ids, K=5)
print("Recommended song IDs and their predicted ratings:")
for song, rating in zip(top_songs, top_ratings):
    print(f"Song ID: {song}, Predicted Rating: {rating:.2f}")

Recommended song IDs and their predicted ratings:
Song ID: 20, Predicted Rating: 8.12
Song ID: 10, Predicted Rating: 7.75
Song ID: 16, Predicted Rating: 6.87
Song ID: 32, Predicted Rating: 6.87
Song ID: 28, Predicted Rating: 6.46


------------------------

# `04` Matrix Factorization using SVD Algorithm

Practice for using `SVD` algorithm implementation from `scikit surprise` library.

## `i` Prepare the Data

- Load the Data into `surprise` Dataset
- Split data into train and test


In [ ]:
data = pd.read_csv("/content/songsDataset.csv")
data.head()

,userID,songID,rating
0,0,90409,5
1,4,91266,1
2,5,8063,2
3,5,24427,4
4,5,105433,4


In [ ]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(data[['userID', 'songID', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

## `ii` Model Initialization

Instantiate two models:
- Model with baselines (biases)
- Model without baselines

**Note**: Use `surprise.prediction_algorithms.matrix_factorization.SVD` (see, [Documentation](https://surprise.readthedocs.io/en/stable/matrix_factorization.html#:~:text=surprise.prediction_algorithms.matrix_factorization.-,SVD,-(n_factors%3D)))

In [ ]:
model_with_baseline = SVD(biased=True)
model_without_baseline = SVD(biased=False)
print("Model with Baseline Biases:", model_with_baseline)
print("Model without Baseline Biases:", model_without_baseline)

Model with Baseline Biases: <surprise.prediction_algorithms.matrix_factorization.SVD object at 0x7cbde905edd0>
Model without Baseline Biases: <surprise.prediction_algorithms.matrix_factorization.SVD object at 0x7cbde8ed0290>


## `iii` Fit each Model on Training Data

In [ ]:
# Biased Model
predictions_with_baseline = model_with_baseline.fit(trainset)

In [ ]:
# Non-Biased Model
predictions_without_baseline = model_without_baseline.fit(trainset)

## `iv` Test both Models on the Testing Data

Compare the errors of the two models using multiple error formulas.

**Note**: Refer to `surprise.accuracy` module (see, [Documentation](https://surprise.readthedocs.io/en/stable/accuracy.html))

In [ ]:
predictions_with_bias = model_with_baseline.test(testset)
accuracy.rmse(predictions_with_bias)
accuracy.mae(predictions_with_bias)

RMSE: 1.4923
MAE:  1.2896


1.289641348887374

In [ ]:
# Non-Biased Model
predictions_without_bias = model_without_baseline.test(testset)
accuracy.rmse(predictions_without_bias)
accuracy.mae(predictions_without_bias)

RMSE: 2.0821
MAE:  1.7331


1.7331336082770852

## `v` Recommend Top $10$ Songs for User $199988$

Is there a difference in recommended songs from the two models?

In [ ]:
def recommend_songs(U, M, R, user_id, user_ids, K=10):
    # Convert user_id to index if needed
    if user_id not in user_ids:
        raise ValueError(f"User ID {user_id} not found in dataset.")

    user_index = np.where(user_ids == user_id)[0][0]  # Map to row index

    # Compute predictions
    R_pred = np.dot(U, M)

    # Get user row
    user_ratings = R[user_index, :]

    # Mask already rated songs
    R_pred[user_index, user_ratings > 0] = -np.inf

    # Get top-K song indices
    top_k_indices = np.argsort(R_pred[user_index, :])[-K:][::-1]
    top_k_ratings = R_pred[user_index, top_k_indices]  # Get predicted ratings

    return top_k_indices, top_k_ratings

# Example usage:
user_ids = np.array(utility_matrix.index)  # Extract user IDs
top_songs, top_ratings = recommend_songs(U, M, utility_matrix.values, user_id=199988, user_ids=user_ids, K=10)

# Print results
print("Top 10 Recommended Songs for User 199988:")
for i, (song, rating) in enumerate(zip(top_songs, top_ratings), 1):
    print(f"\t- Top {i} Song: {song} (Predicted Rating: {rating:.6f})")


Top 10 Recommended Songs for User 199988:
	- Top 1 Song: 20 (Predicted Rating: 8.123471)
	- Top 2 Song: 10 (Predicted Rating: 7.749010)
	- Top 3 Song: 16 (Predicted Rating: 6.869100)
	- Top 4 Song: 32 (Predicted Rating: 6.865494)
	- Top 5 Song: 28 (Predicted Rating: 6.462294)
	- Top 6 Song: 14 (Predicted Rating: 6.403175)
	- Top 7 Song: 46 (Predicted Rating: 6.374434)
	- Top 8 Song: 9 (Predicted Rating: 6.285680)
	- Top 9 Song: 2 (Predicted Rating: 6.245649)
	- Top 10 Song: 30 (Predicted Rating: 6.096276)


In [ ]:
all_items = set(utility_matrix.columns)
rated_items = set(utility_matrix.loc[199988][utility_matrix.loc[199988] > 0].index)
unrated_items = all_items - rated_items
predictions_with_bias = [(item, predictions_with_baseline.predict(199988, item).est) for item in unrated_items]
predictions_without_bias = [(item, predictions_without_baseline.predict(199988, item).est) for item in unrated_items]

In [ ]:
# Biased Model
top_10_with_bias = sorted(predictions_with_bias, key=lambda x: x[1], reverse=True)[:10]
print("Top 10 Recommended Songs for User 199988 (Biased Model):")
for i, (song, rating) in enumerate(top_10_with_bias, 1):
    print(f"\t- Top {i} Song: {song} (Predicted Rating: {rating:.2f})")

Top 10 Recommended Songs for User 199988 (Biased Model):
	- Top 1 Song: 105433 (Predicted Rating: 4.66)
	- Top 2 Song: 92881 (Predicted Rating: 4.61)
	- Top 3 Song: 134732 (Predicted Rating: 4.59)
	- Top 4 Song: 126757 (Predicted Rating: 4.49)
	- Top 5 Song: 71582 (Predicted Rating: 4.45)
	- Top 6 Song: 112023 (Predicted Rating: 4.45)
	- Top 7 Song: 122065 (Predicted Rating: 4.32)
	- Top 8 Song: 52611 (Predicted Rating: 4.29)
	- Top 9 Song: 62954 (Predicted Rating: 4.27)
	- Top 10 Song: 79622 (Predicted Rating: 4.26)


In [ ]:
# Non-Biased Model
top_10_without_bias = sorted(predictions_without_bias, key=lambda x: x[1], reverse=True)[:10]
print("Top 10 Recommended Songs for User 199988 (Non-Biased Model):")
for i, (song, rating) in enumerate(top_10_without_bias, 1):
    print(f"\t- Top {i} Song: {song} (Predicted Rating: {rating:.2f})")

Top 10 Recommended Songs for User 199988 (Non-Biased Model):
	- Top 1 Song: 92881 (Predicted Rating: 3.30)
	- Top 2 Song: 40712 (Predicted Rating: 3.12)
	- Top 3 Song: 119103 (Predicted Rating: 3.11)
	- Top 4 Song: 86341 (Predicted Rating: 3.08)
	- Top 5 Song: 8063 (Predicted Rating: 2.94)
	- Top 6 Song: 55622 (Predicted Rating: 2.36)
	- Top 7 Song: 122065 (Predicted Rating: 2.29)
	- Top 8 Song: 94604 (Predicted Rating: 2.19)
	- Top 9 Song: 17029 (Predicted Rating: 2.16)
	- Top 10 Song: 74640 (Predicted Rating: 2.08)


----------------------------------------------

$$ Wish \space you \space all \space the \space best \space ♡ $$
$$ Mahmoud \space Shawqi $$